Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [92]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
from random import random, choice, randint
from copy import deepcopy, copy
from dataclasses import dataclass
from pprint import pprint
import numpy as np


In [93]:
POPULATION_SIZE = 30 #mi
OFFSPRING_SIZE = 20 #lambda #creating 20 new individuals
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15
NUM_GAMES = 10

## The *Nim* and *Nimply* classes

In [94]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [95]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [97]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [98]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [100]:
def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = choice(spicy_moves)
    return ply


## My strategy

In [103]:
def adaptive(state: Nim, ind: Individual) -> Nimply:
    """A strategy that can adapt its parameters"""
    
     # numero di righe
    num_rows = state._rows

    # numero di stecchini
    num_sticks = sum(state._rows)

    if num_rows %2 == 0:
       if num_sticks %2 == 0:
           


    

    
    



In [96]:
@dataclass
class Individual:
    fitness: tuple 
    genotype: list[float] 

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(individual: Individual) -> Individual:
    offspring = copy(individual)
    position = randint(0, NUM_SETS-1)
    offspring.genotype[position] = not offspring.genotype[position]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness = None, genotype = ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    
    return offspring

In [ ]:
def fitness(phenotype: Individual):
    total_wins = 0

    strategy = (adaptive, optimal)

    for _ in range (NUM_GAMES):
        nim = Nim(5)
        player = 0
        while nim:
            #mossa da fare
            #controllo che strategia sto usando 
            if strategy[player] == adaptive:
                ply = strategy[player](nim, phenotype)
            else:
                ply = strategy[player](nim)
            # fa la mossa
            nim.nimming(ply)

            #cambio giocatore
            player = 1 - player
        
        if player == 0:
            total_wins += 1

    return total_wins

In [ ]:
#initial population
population = [Individual(fitness = None, genotype=[0, 0]) for _ in range (POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genotype)

In [ ]:
for generation in range(100):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:   # self-adapt mutation probability
            #do mutation # add more clever mutation
            parent = select_parent(population)
            child = mutate(p)
        else:
            #do xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            child = one_cut_xover(p1, p2)
            
        offspring.append(child)

    population = offspring

    for i in population:
        i.fitness = fitness(i.genotype)
        
    population.extend(offspring)
    population.sort(key = lambda i: i.fitness, reverse = True)

    #survival selection
    population = population[:POPULATION_SIZE]
    print(population[0].fitness)

## Oversimplified match

In [101]:

logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 5 4 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=7)
INFO:root:status: <1 3 5 4 2>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=5)
INFO:root:status: <1 3 0 4 2>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 0 4 1>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 0 1 1>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 2 0 1 1>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 2 0 0 1>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 1 0 0 1>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 1 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 0 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0